# Doporučovací systém

In [3]:
import datetime
import pandas as pd
from sklearn.tree import DecisionTreeClassifier  
from joblib import load

In [ ]:
# Fetch the current time
current_time = datetime.datetime.now()

# Prepare the input features based on the current time
decimal_time = current_time.hour + current_time.minute / 60.0
day_of_week = current_time.weekday()  # Monday is 0 and Sunday is 6

# Create a DataFrame for the input features (adjust column names as needed)
input_features = pd.DataFrame([[decimal_time, day_of_week]],
                              columns=['DecimalTime', 'DayOfWeek'])

# Predict using the trained model (clf_best)
predicted_camera = clf_best.predict(input_features)

# If your model's target variable was encoded with LabelEncoder, decode the prediction
predicted_camera_label = le.inverse_transform(predicted_camera)

print(f"Predicted camera for the current time ({current_time}): {predicted_camera_label[0]}")

### Načtení modelu

In [4]:
decision_tree_clf = load('model/decision_tree.joblib')

### Zisk aktuálního času a dne ze systému

In [17]:
current_time = datetime.datetime.now()
print(f'Aktuální čas je {current_time}')

decimal_time = current_time.hour + current_time.minute / 60.0
day_of_week = current_time.weekday() 
input_features = pd.DataFrame([[decimal_time, day_of_week]],
                              columns=['DecimalTime', 'DayOfWeek'])

Aktuální čas je 2024-04-08 14:04:52.739255


### Doporučení kamery

In [15]:
predicted_camera = decision_tree_clf.predict(input_features)
print(f'Doporučená kamera je {predicted_camera[0]}.')

Doporučená kamera je 6.
